In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
!pip install langchain
!pip install faiss-cpu
!pip install pyttsx3
!pip install transformers
!pip install torch
!pip install sentence-transformers



Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install pymupdf




Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install SpeechRecognition
!pip install pyaudio


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
import fitz  # PyMuPDF

def load_pdf(path):
    doc = fitz.open("DOC-20250418-WA0002_250727_210846.pdf")
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_path = "DOC-20250418-WA0002_250727_210846.pdf"  # Make sure this file is in the same folder as your notebook
raw_text = load_pdf(pdf_path)
print("Text length:", len(raw_text))


Text length: 955648


In [6]:
!pip install sentence-transformers
!pip install faiss-cpu


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define your chunking method
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Each chunk will have ~500 characters
    chunk_overlap=100  # Overlap between chunks (helps for continuity)
)

chunks = text_splitter.split_text(raw_text)
print(f"Total chunks created: {len(chunks)}")


Total chunks created: 2252


In [8]:

import torch
import numpy as np
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

def search_chunks(query, chunks, top_k=3, threshold=0.6):
    # Convert query and chunks to embeddings
    query_embedding = model.encode(query, convert_to_tensor=True)
    chunk_embeddings = model.encode(chunks, convert_to_tensor=True)

    # Compute cosine similarities
    cosine_scores = util.pytorch_cos_sim(query_embedding, chunk_embeddings)[0]

    # Find best matching chunk
    top_result = torch.topk(cosine_scores, k=1)
    best_score = top_result.values[0].item()
    best_chunk = chunks[top_result.indices[0]]

    # Check threshold
    if best_score < threshold:
        return "This question is not covered in the textbook."
    else:
        return best_chunk


In [9]:
def generate_answer(question, pdf_chunks):
    relevant_chunks = search_pdf_chunks(question, pdf_chunks)
    
    if not relevant_chunks or len(relevant_chunks[0]) < 20:  # No meaningful match
        return "This information is not available in the textbook."

    # Pass only relevant PDF content to the model
    context = " ".join(relevant_chunks)
    answer = local_llm_generate(question, context)
    return answer


In [10]:

import re

def get_exact_paragraph(query, chunks, threshold=0.6):
    """Finds and returns the most relevant paragraph from the PDF."""
    best_chunk = search_chunks(query, chunks, threshold=threshold)
    if best_chunk == "This question is not covered in the textbook.":
        return best_chunk

    # Split into paragraphs and find best match
    paragraphs = [p.strip() for p in re.split(r'\n\s*\n', best_chunk) if p.strip()]
    if not paragraphs:
        return best_chunk

    query_embedding = model.encode(query, convert_to_tensor=True)
    para_embeddings = model.encode(paragraphs, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(query_embedding, para_embeddings)[0]

    best_para_idx = torch.argmax(cosine_scores).item()
    return paragraphs[best_para_idx]


In [11]:
import ollama
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pyttsx3

# ------------------------------
# 1. Extract text from PDF
# ------------------------------
def extract_pdf_text(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

# ------------------------------
# 2. Split text into chunks
# ------------------------------
def split_into_chunks(text, chunk_size=500):
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# ------------------------------
# 3. Create FAISS index
# ------------------------------
def create_faiss_index(chunks, model):
    embeddings = model.encode(chunks)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))
    return index, embeddings

# ------------------------------
# 4. Find top N best chunks
# ------------------------------
def find_top_chunks(question, chunks, model, index, top_n=3):
    q_embedding = model.encode([question])
    distances, indices = index.search(np.array(q_embedding), k=top_n)
    return [chunks[i] for i in indices[0]]

# ------------------------------
# 5. Ask TinyLLaMA using Ollama
# ------------------------------
def ask_local_llm(context, question):
    prompt = f"""You are a helpful AI tutor. Based only on the textbook content below, answer the question clearly and accurately.

Text:
{context}

Question:
{question}

Answer:"""
    response = ollama.chat(
        model='tinyllama',
        messages=[{'role': 'user', 'content': prompt}]
    )
    return response['message']['content']

# ------------------------------
# 6. Speak the answer (TTS)
# ------------------------------
def speak_text(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

# ------------------------------
# MAIN SCRIPT (Interactive Chat with Voice)
# ------------------------------
if __name__ == "__main__":
    pdf_path = r"DOC-20250418-WA0002_250727_210846.pdf"  # <-- Your PDF file path

    print("📖 Extracting PDF text...")
    full_text = extract_pdf_text(pdf_path)
    if not full_text.strip():
        print("❌ Could not extract text from the PDF.")
        exit()

    print("🔄 Splitting into chunks...")
    chunks = split_into_chunks(full_text)
    print(f"✅ PDF split into {len(chunks)} chunks.")

    print("🔍 Creating FAISS index...")
    model = SentenceTransformer('all-MiniLM-L6-v2')
    index, embeddings = create_faiss_index(chunks, model)
    print("✅ Ready! Ask any question (type 'exit' to quit).")

    # Interactive Loop
    while True:
        question = input("\n❓ Your Question: ")
        if question.lower() in ["exit", "quit", "bye"]:
            print("👋 Exiting AI Tutor. Goodbye!")
            break

        print("🔍 Searching PDF for relevant content...")
        top_chunks = find_top_chunks(question, chunks, model, index, top_n=3)
        combined_context = "\n\n".join(top_chunks)

        print("🤖 Generating answer...\n")
        answer = ask_local_llm(combined_context, question)

        print(f"💬 AI Tutor Answer:\n{answer}\n")

        # Speak answer
        print("🔊 Speaking answer...")
        speak_text(answer)


📖 Extracting PDF text...
🔄 Splitting into chunks...
✅ PDF split into 302 chunks.
🔍 Creating FAISS index...
✅ Ready! Ask any question (type 'exit' to quit).
🔍 Searching PDF for relevant content...
🤖 Generating answer...

💬 AI Tutor Answer:
Decentralization refers to the practice of ensuring that all resources, information, and activities are distributed without a central authority or control system. The most well-known examples of decentralized systems include Bitcoin and Ethereum. In this context, the term is often used interchangeably with "blockchain" as in "blockchain-based decentralized applications."

In order to create an effective decentralized system, several key principles must be followed, including:

1. Anonymity: Users should have the freedom to choose how their identities are revealed or hidden. This ensures that all users can participate equally and that any given user's actions cannot be tied back to them.

2. Consensus: The system should be designed so that all validato

In [15]:
import ollama
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pyttsx3
import speech_recognition as sr

# ------------------------------
# 1. Extract text from PDF
# ------------------------------
def extract_pdf_text(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

# ------------------------------
# 2. Split text into chunks
# ------------------------------
def split_into_chunks(text, chunk_size=500):
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# ------------------------------
# 3. Create FAISS index
# ------------------------------
def create_faiss_index(chunks, model):
    embeddings = model.encode(chunks)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))
    return index, embeddings

# ------------------------------
# 4. Find top N best chunks
# ------------------------------
def find_top_chunks(question, chunks, model, index, top_n=3):
    q_embedding = model.encode([question])
    distances, indices = index.search(np.array(q_embedding), k=top_n)
    return [chunks[i] for i in indices[0]]

# ------------------------------
# 5. Ask TinyLLaMA using Ollama
# ------------------------------
def ask_local_llm(context, question):
    prompt = f"""You are a helpful AI tutor. Based only on the textbook content below, answer the question clearly and accurately.

Text:
{context}

Question:
{question}

Answer:"""
    response = ollama.chat(
        model='tinyllama',
        messages=[{'role': 'user', 'content': prompt}]
    )
    return response['message']['content']

# ------------------------------
# 6. Speak the answer (TTS)
# ------------------------------
def speak_text(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

# ------------------------------
# 7. Listen for voice question
# ------------------------------
def listen_question():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("🎤 Listening... Please speak your question.")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try:
        question = recognizer.recognize_google(audio)
        print(f"❓ You asked: {question}")
        return question
    except sr.UnknownValueError:
        print("❌ Sorry, I couldn't understand. Please try again.")
        return None
    except sr.RequestError:
        print("❌ Voice recognition service is unavailable.")
        return None

# ------------------------------
# MAIN SCRIPT
# ------------------------------
if __name__ == "__main__":
    pdf_path = r"DOC-20250418-WA0002_250727_210846.pdf" # <-- Your PDF file

    print("📖 Extracting PDF text...")
    full_text = extract_pdf_text(pdf_path)
    if not full_text.strip():
        print("❌ Could not extract text from the PDF.")
        exit()

    print("🔄 Splitting into chunks...")
    chunks = split_into_chunks(full_text)
    print(f"✅ PDF split into {len(chunks)} chunks.")

    print("🔍 Creating FAISS index...")
    model = SentenceTransformer('all-MiniLM-L6-v2')
    index, embeddings = create_faiss_index(chunks, model)
    print("✅ Ready! Say 'exit' to quit.")

    while True:
        question = listen_question()
        if not question:
            continue
        if question.lower() in ["exit", "quit", "bye"]:
            print("👋 Exiting AI Tutor. Goodbye!")
            speak_text("Goodbye! Exiting AI Tutor.")
            break

        print("🔍 Searching PDF for relevant content...")
        top_chunks = find_top_chunks(question, chunks, model, index, top_n=3)
        combined_context = "\n\n".join(top_chunks)

        print("🤖 Generating answer...\n")
        answer = ask_local_llm(combined_context, question)

        print(f"💬 AI Tutor Answer:\n{answer}\n")
        speak_text(answer)




📖 Extracting PDF text...
🔄 Splitting into chunks...
✅ PDF split into 302 chunks.
🔍 Creating FAISS index...
✅ Ready! Say 'exit' to quit.
🎤 Listening... Please speak your question.
❌ Sorry, I couldn't understand. Please try again.
🎤 Listening... Please speak your question.
❓ You asked: Who is the author of
🔍 Searching PDF for relevant content...
🤖 Generating answer...

💬 AI Tutor Answer:
Mastering Blockchain Second Edition by Imran Bashir is written by a software development, solution architecture, infraestructure management, and IT service management professional who has experience working in large-scale IT projects in the public sector before moving to the financial services industry. The author is also a member of the Institute of Electrical and Electronics Engineers (IEEE) and British Computer Society (BCS).

🎤 Listening... Please speak your question.
❌ Sorry, I couldn't understand. Please try again.
🎤 Listening... Please speak your question.
❌ Sorry, I couldn't understand. Please tr

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# 1. Prepare test Q&A pairs (you can add more questions)
qa_pairs = [
    ("What is blockchain?", "Blockchain is a decentralized and distributed digital ledger used to record transactions securely."),
    ("What is decentralization in blockchain?", "Decentralization means that no single entity controls the blockchain; instead, it is managed by a network of participants."),
    ("What is a block in blockchain?", "A block is a data structure containing a list of transactions, a timestamp, and a cryptographic hash of the previous block."),
    ("What is a hash in blockchain?", "A hash is a fixed-length alphanumeric string generated by a cryptographic algorithm that uniquely represents data."),
    ("What is a smart contract?", "A smart contract is a self-executing program stored on the blockchain that runs when predefined conditions are met."),
]

# 2. Load sentence transformer for semantic similarity
similarity_model = SentenceTransformer('all-MiniLM-L6-v2')

# 3. Function to compute similarity
def get_similarity(ans1, ans2):
    emb1 = similarity_model.encode([ans1])
    emb2 = similarity_model.encode([ans2])
    return cosine_similarity(emb1, emb2)[0][0]

# 4. Accuracy Test
correct = 0
for i, (question, expected) in enumerate(qa_pairs, 1):
    # Use your AI tutor function to answer (replace 'ask_local_llm' with your function)
    top_chunks = find_top_chunks(question, chunks, model, index, top_n=2)
    context = "\n".join(top_chunks)
    ai_answer = ask_local_llm(context, question)  # Your existing code's answer
    sim = get_similarity(ai_answer, expected)

    print(f"Q{i}: {question}")
    print(f"AI Answer: {ai_answer}")
    print(f"Expected: {expected}")
    print(f"Similarity Score: {sim:.2f}\n")

    if sim > 0.7:
        correct += 1

accuracy = (correct / len(qa_pairs)) * 100
print(f"✅ Final Accuracy: {accuracy:.2f}%")


Q1: What is blockchain?
AI Answer: Blockchain is an innovative and decentralized distributed ledger technology (DLT) that allows for secure and immutable transactions between participants in a network with no central point of control. Unlike traditional banking systems, where information is stored on a single server or computer, blockchain uses a peer-to-peer network to store data securely. This network is based on cryptographic protocols to ensure security and fairness for all participants. The blockchain is a digital ledger that records transactions and provides tamper-evident guarantees of the data stored within it. In practice, blockchain technology has wide-ranging applications in various industries, including but not limited to finance, healthcare, government, media, law, and arts. The concept of blockchain is an advancement from traditional banking systems, which have required banks or central clearing houses as a trusted authority for the database. As decentralization is one of